In [6]:
# import requried dependencies
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# read the data
df = pd.read_csv(r"C:\Users\bcrbb\Downloads\AI\YouSpo.csv", low_memory=False)[:20079]

In [7]:
# remove duplicates
# df.drop_duplicates(subset=["song", "artist"], inplace=True)
df.drop_duplicates(subset=["song"], inplace=True)
df.dropna(subset=["song", "link"], inplace=True)  # Ensure no NaN values in key columns

# Drop the non-required columns
df = df.drop(df.columns[4:], axis=1)
# Removing space from "Artist Name" column
df["artist"] = df["artist"].str.replace(" ", "")

In [10]:
# Combine necessary columns for similarity calculation, typically 'song' and 'artist'
df['combined'] = df.apply(lambda value: " ".join(value.astype("str")), axis=1)
# convert the text data into a format that can be analyzed
vectorizer = CountVectorizer()
#Transforms the 'combined' column into a numeric matrix 
vectorized = vectorizer.fit_transform(df['combined'])
#Computes the cosine similarity between all pairs of songs based on their vectorized text representations
similarities = cosine_similarity(vectorized)
# Create the similarity DataFrame indexed by song names
df_tmp = pd.DataFrame(similarities, index=df['song'], columns=df['song'])

print(similarities)

# print(df_tmp)

[[1.         0.72057669 0.57265629 ... 0.38461538 0.41812101 0.37062466]
 [0.72057669 1.         0.59603956 ... 0.40032038 0.43519414 0.38575837]
 [0.57265629 0.59603956 1.         ... 0.31814238 0.34585723 0.3678836 ]
 ...
 [0.38461538 0.40032038 0.31814238 ... 1.         0.66899361 0.59299945]
 [0.41812101 0.43519414 0.34585723 ... 0.66899361 1.         0.64465837]
 [0.37062466 0.38575837 0.3678836  ... 0.59299945 0.64465837 1.        ]]


In [11]:
import tkinter as tk
from tkinter import ttk, messagebox, Listbox
import webbrowser
import pandas as pd
import re 


def open_link(url):
    webbrowser.open(url, new=2)

def show_recommendations(entry_widget, tree_view, root):
    input_song = entry_widget.get().strip()  # Normalize the input
    if not input_song:
        messagebox.showerror("Error", "Please enter a song.")
        return

    if input_song in df_tmp.index:
        song_data = df_tmp.loc[input_song]
        if isinstance(song_data, pd.Series):
            top_indices = song_data.nlargest(11).index
            top_indices = [idx for idx in top_indices if idx != input_song][:10]
            recommendations = df.loc[df['song'].isin(top_indices), ['song', 'link', 'original_artist']]
            tree_view.delete(*tree_view.get_children())
            for idx, (song, link, artist) in enumerate(zip(recommendations['song'], recommendations['link'], recommendations['original_artist']), start=1):
                tree_view.insert('', 'end', values=(idx, song, artist, link))
            entry_widget.focus_set()  # Set focus back to the entry widget after updating the tree
    else:
        messagebox.showerror("Error", "Song not found in the database.")


def update_suggestions(event, listbox, entry_widget):
    typed_text = entry_widget.get().strip()
    if not typed_text:
        listbox.delete(0, tk.END)
        return

    # Escape any special characters in the typed text to avoid regex issues
    escaped_text = re.escape(typed_text)

    # Use the escaped text for a safe search
    suggestions = df[df['song'].str.contains(escaped_text, case=False, na=False)]
    listbox.delete(0, tk.END)
    for _, row in suggestions.iterrows():
        listbox.insert(tk.END, f"{row['song']} → {row['artist']}")

def fill_entry_from_listbox(listbox, entry_widget, root):
    try:
        selection = listbox.get(listbox.curselection())
        song_name = selection.split(' → ')[0]
        entry_widget.delete(0, tk.END)
        entry_widget.insert(0, song_name.strip())
        entry_widget.focus_set()  # Set focus back to the entry widget
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

def on_click(event, tree):
    region = tree.identify("region", event.x, event.y)
    if region == "cell":
        col = tree.identify_column(event.x)
        if col == "#4":  # Link column
            item = tree.selection()[0]
            link = tree.item(item, "values")[3]
            open_link(link)

def sort_column(tree, col, reverse):
    l = [(tree.set(k, col), k) for k in tree.get_children('')]
    l.sort(key=lambda x: int(x[0]) if col == 'No' else x[0], reverse=reverse)
    for index, (val, k) in enumerate(l):
        tree.move(k, '', index)
    tree.heading(col, command=lambda: sort_column(tree, col, not reverse))

def initialize_app(root):
    def confirm_exit():
        if messagebox.askyesno("Confirm Exit", "Are you sure you want to exit?"):
            root.destroy()

    def goodbye_message():
        messagebox.showinfo("Goodbye", "Thank you for using the Song Recommender. Have a great day!")


    root.title("Song Recommender")
    root.geometry('1000x800')
    root.configure(bg='#D7E3FC')

    style = ttk.Style()
    style.theme_use('clam')
    style.configure('TFrame', background='#D7E3FC')
    style.configure('TLabel', background='#D7E3FC', foreground='black', font=('Arial', 20, 'bold'))
    style.configure('TButton', background='#E6E6FA', foreground='black')
    style.configure('TEntry', fieldbackground='#E6E6FA', foreground='black')

    frame = ttk.Frame(root, padding="10")
    frame.pack(padx=10, pady=10, fill='both', expand=True)
    title_label = ttk.Label(frame, text="Top 10 Recommend Songs", style='TLabel')
    title_label.pack(pady=10)
    
    tree = ttk.Treeview(frame, columns=('No', 'Song', 'Artist', 'Link'), show='headings', height=15)
    tree.heading('No', text='No', command=lambda: sort_column(tree, 'No', False))
    tree.heading('Song', text='Song', command=lambda: sort_column(tree, 'Song', False))
    tree.heading('Artist', text='Artist', command=lambda: sort_column(tree, 'Artist', False))
    tree.heading('Link', text='Link')
    tree.column('No', width=10, anchor='center')
    tree.column('Song', width=250, anchor='w')
    tree.column('Artist', width=200, anchor='w')
    tree.column('Link', width=250, anchor='w')
    tree.pack(pady=20, fill='x', expand=True)
    tree.bind("<ButtonRelease-1>", lambda event: on_click(event, tree))

    input_frame = ttk.Frame(frame)
    input_frame.pack(expand=True)
    song_entry = ttk.Entry(input_frame, width=50)
    song_entry.pack(side='left', padx=5)
    song_entry.focus_force()

    submit_button = ttk.Button(input_frame, text="Get Recommendations", command=lambda: show_recommendations(song_entry, tree, root))
    submit_button.pack(side='left', pady=5, padx=5)

    suggestion_frame = ttk.Frame(frame)
    suggestion_frame.pack(expand=True)
    suggestion_listbox = Listbox(suggestion_frame, width=75)
    suggestion_listbox.pack(side='left', padx=10)
    song_entry.bind("<KeyRelease>", lambda event: update_suggestions(event, suggestion_listbox, song_entry))
    suggestion_listbox.bind("<Double-1>", lambda event: fill_entry_from_listbox(suggestion_listbox, song_entry, root))

    exit_button = ttk.Button(frame, text="Exit",command=confirm_exit)
    exit_button.pack(pady=10)

    root.protocol("WM_DELETE_WINDOW", confirm_exit)
    song_entry.bind("<Return>", lambda event: show_recommendations(song_entry, tree, root))  # Bind Enter key
    root.mainloop()
    goodbye_message()
  
def main():
    root = tk.Tk()
    root.withdraw()  # Hide the main window initially
    messagebox.showinfo("Welcome to Song Recommender", "Welcome to the Top 10 Song Recommendation System!\nClick OK to start.")
    root.deiconify()  # Show the main window after the welcome message
    initialize_app(root)

if __name__ == "__main__":
    main()
